# Load data

In [1]:
from pymatgen import Structure
from monty.serialization import loadfn

data = loadfn('data.json')
train_structures = [d['structure'] for d in data]
train_energies = [d['outputs']['energy'] for d in data]
train_forces = [d['outputs']['forces'] for d in data]

# Setup the initial weights for training (If not, the weights for energy and force will be both equal to 1)

In [2]:
import numpy as np
from mlearn.data.processing import pool_from, convert_docs

train_pool = pool_from(train_structures, train_energies, train_forces)
_, df = convert_docs(train_pool)

weights = np.ones(len(df['dtype']), )

# set the weights for energy equal to 100
weights[df['dtype'] == 'energy'] = 100

# Set up the SNAP and train

In [3]:
from mlearn.describer.atomic_describer import BispectrumCoefficients
from mlearn.model.linear_model import LinearModel
from mlearn.potential.snap import SNAPotential

element_profile = {'Mo': {'r': 0.5, 'w': 1}}
describer = BispectrumCoefficients(rcutfac=5.0, twojmax=6, element_profile=element_profile, 
                                   quadratic=False, pot_fit=True)
model = LinearModel(describer=describer)
snap = SNAPotential(model=model)
snap.train(train_structures, train_energies, train_forces, weights=weights)

# Lattice constant, Elastic constant

In [4]:
from mlearn.potential.lammps.calcs import LatticeConstant

conventional_cell = Structure.from_file('conventional.cif')
lc_calculator = LatticeConstant(ff_settings=snap)
a, b, c = lc_calculator.calculate([conventional_cell])[0]
print('Lattice a: {}, Lattice b: {}, Lattice c: {}'.format(a, b, c))

Lattice a: 3.30415747249494, Lattice b: 3.30415747249494, Lattice c: 3.30415747249494


In [5]:
from mlearn.potential.lammps.calcs import ElasticConstant

ec_calculator = ElasticConstant(ff_settings=snap, lattice='bcc', alat=3.106)
C11, C12, C44, bulk_modulus = ec_calculator.calculate()
print('C11: {}, C12: {}, C44: {}, bulk modulus: {}'.format(C11, C12, C44, bulk_modulus))

C11: 277.785784355365, C12: 77.0089201323572, C44: 80.7990295693602, bulk modulus: 143.934541540431


# Load model from parameters files

In [7]:
snap_loaded = SNAPotential.from_config(param_file='SNAPotential.snapparam', coeff_file='SNAPotential.snapcoeff')

# Energy, force, stress prediction